In [58]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
spark = SparkSession.builder \
    .appName("Analysis") \
    .getOrCreate()

In [59]:
table1= (
    spark.read.csv(
        "../dataset/New/final_set2.csv", 
        header=True,
        inferSchema=True
    )
    .withColumn("good", f.when(f.col("90>=day_contract") == 0, 1).otherwise(0))
    .withColumn("bad", f.when(f.col("90>=day_contract") == 1, 1).otherwise(0))
    .select(
        "FAC NO",
        "BRN CD",
        "PROCODE",
        "CUST_PERMRES",
        "GUAR_PERMRES",
        "EXP_CD",
        "DSCR_C",
        "connex_date",
        "DPD",
        "90>=day_contract",
        "good",
        "bad"
    )
# .withColumn("GUAR_PERMRES", f.when((f.col("GUAR_PERMRES").isNull())&(f.col("DSCR_C").isNotNull()), "No Guaranter").otherwise(f.col("GUAR_PERMRES")))
.filter(f.col("90>=day_contract").isNotNull())
.filter(f.col("connex_date").between("2022-04-01","2023-04-01"))
.withColumn("Product",f.when(f.col("PROCODE")==502,"Registered 3w").otherwise("Cash in hand"))
.withColumn("CUST_PERMRES",
            f.when(f.col("CUST_PERMRES")=="With Parents/spouse",
                   "With Parents/Spouse"
                  ).otherwise(f.col("CUST_PERMRES")))
.withColumn("GUAR_PERMRES",
            f.when(f.col("GUAR_PERMRES")=="With Parents/spouse",
                   "With Parents/Spouse"
                  ).otherwise(f.col("GUAR_PERMRES")))
.withColumn("GUAR_PERMRES",(f.when(
    (f.col("GUAR_PERMRES").isNull())
    &(f.col("EXP_CD").isNotNull())
    &(f.col("DSCR_C").isNotNull()),"No Guaranter").otherwise(f.col("GUAR_PERMRES")))
)  
)
table2=(
    spark.read.csv(
        "../dataset/New/detail_report.csv", 
        header=True,
        inferSchema=True
    ).select(
        "ACCOUNT NUMBER",
        "Conex Date",
        "Portfolio",
        "TOTAL DUE",
        "TOTAL COLLECTION",
        "Collection Ratio"
    )
    .withColumnRenamed("ACCOUNT NUMBER","FAC NO")
    .filter(f.col("Conex Date").between("2022-04-01","2023-04-01"))
    )
table3=(
    spark.read.csv(
        "../dataset/New/finalcrib.csv", 
        header=True,
        inferSchema=True
    ).select(
        "Fac",
        "CRIB score"
    )
    .withColumnRenamed("Fac","FAC NO")
    
    )


In [60]:
table3.show()

+-------------------+----------+
|             FAC NO|CRIB score|
+-------------------+----------+
|'041600794120050202|  -400.000|
|'006100798148050201|  -400.000|
|'011500798116050201|  -400.000|
|'011500044709050202|  -400.000|
|'013800796663050202|  -400.000|
|'013800796663050201|  -400.000|
|'007000797964050801|  -400.000|
|'017500798051050201|  -400.000|
|'005000413648050801|    50.950|
|'006800796830050202|  -400.000|
|'002900711921050801|  -400.000|
|'005200797575050202|  -400.000|
|'005000622866050802|  -400.000|
|'000600798195050201|  -400.000|
|'013900797777050201|  -400.000|
|'003400798039050202|  -400.000|
|'005000797918050201|  -400.000|
|'000800798010050201|  -400.000|
|'000300798074050201|  -400.000|
|'000600798054050201|  -400.000|
+-------------------+----------+
only showing top 20 rows



In [61]:
master_table=(
    table1
    .join(table2,"FAC NO","left")
    .join(table3,"FAC NO","left")
    .filter(f.col("Conex Date").between("2022-04-01","2023-04-01"))
)

In [62]:
master_table.show()

+-------------------+------+-------+--------------------+-------------------+------+---------+-----------+---+----------------+----+---+-------------+----------+---------+---------+----------------+----------------+----------+
|             FAC NO|BRN CD|PROCODE|        CUST_PERMRES|       GUAR_PERMRES|EXP_CD|   DSCR_C|connex_date|DPD|90>=day_contract|good|bad|      Product|Conex Date|Portfolio|TOTAL DUE|TOTAL COLLECTION|Collection Ratio|CRIB score|
+-------------------+------+-------+--------------------+-------------------+------+---------+-----------+---+----------------+----+---+-------------+----------+---------+---------+----------------+----------------+----------+
|'014200183766050801|   142|    508|  With Billing Proof|       No Guaranter| 32.04|3.2785542| 2022-06-29| 11|               0|   1|  0| Cash in hand|2022-06-29|  143,315|  445,127|         513,619|         274.17%|  -400.000|
|'041200813926050803|   412|    508|                Rent|               Rent|  31.2|5.345211

In [63]:
master_table.select("90>=day_contract").distinct().show()

+----------------+
|90>=day_contract|
+----------------+
|               0|
|               1|
+----------------+



In [64]:
master_table_pd=master_table.toPandas()

In [65]:
master_table_pd.to_csv("../output/last/master_table3.csv")

## Analysis

In [66]:
master_table_new= (
    spark.read.csv(
        "../dataset/New/final_set2.csv", 
        header=True,
        inferSchema=True
    )
)

In [67]:
master_table_new.count()

14927

# Taking good/bad as target variable(90 DPD Contracts)

### Product wise

In [484]:
master_table_new_product=(
     master_table_new
)
Product_wise= (
        master_table_new_product
        .groupBy("Product")
        .agg(
            f.sum("good").alias("good"),
            f.sum("bad").alias("bad")
        )
    )
total_good=(
        Product_wise
        .agg(f.sum("good").alias("good"))
        .collect()[0]['good']
    )
total_bad= (
        Product_wise
        .agg(f.sum("bad").alias("bad"))
        .collect()[0]['bad']
    )

Product_wise=(
    Product_wise
    .withColumn(
        "Total_rate",
        ((f.col("good")+f.col("bad"))/(total_good+total_bad))
        )
    .withColumn(
        "good_rate",
        f.col("good")/total_good    
    )
    .withColumn(
        "bad_rate",
        f.col("bad")/total_bad   
    )
    .withColumn(
        "odd",
        f.col("good")/f.col("bad")  
    )
    .withColumn(
    "IV",
    (
        ((f.col("good") / total_good) - (f.col("bad") / total_bad)) *
        f.ln((f.col("good") / total_good) / (f.col("bad") / total_bad))
    )
)
    .withColumn(
       "good_rate_of_sub_cat",
    f.col("good") / (f.col("good") + f.col("bad"))
    )
    .orderBy("good_rate_of_sub_cat")
)

# Iv_value_Product_wise= 0.029287+0.058048
Iv_value_Product_wise= 0.031372+0.048661

In [485]:
master_table_new_product.select("Product").distinct().show()

+-------------+
|      Product|
+-------------+
|Registered 3w|
| Cash in hand|
+-------------+



In [486]:
total_bad

1374

In [487]:
total_good

6278

In [488]:
Product_wise.toPandas()

,Product,good,bad,Total_rate,good_rate,bad_rate,odd,IV,good_rate_of_sub_cat
0,Registered 3w,3376,928,0.562467,0.537751,0.6754,3.637931,0.031372,0.784387
1,Cash in hand,2902,446,0.437533,0.462249,0.3246,6.506726,0.048661,0.866786


In [489]:
Product_wise.toPandas().to_csv("../output/last/product_wise_final.csv")

PermissionError: [Errno 13] Permission denied: '../output/last/product_wise_final.csv'

In [490]:
Iv_value_Product_wise

0.08003299999999999

#### CUST_PERMRES

In [491]:
master_table_new_custer_pre=(
     master_table_new
    .filter(f.col("CUST_PERMRES").isNotNull())
)
CUST_PERMRES_wise= (
       master_table_new_custer_pre

    .groupBy("CUST_PERMRES")
        .agg(
            f.sum("good").alias("good"),
            f.sum("bad").alias("bad")
        )
    )
total_good_CUST_PERMRES_wise=(
       CUST_PERMRES_wise
        .agg(f.sum("good").alias("good"))
        .collect()[0]['good']
    )
total_bad_CUST_PERMRES_wise= (
        CUST_PERMRES_wise
        .agg(f.sum("bad").alias("bad"))
        .collect()[0]['bad']
    )

CUST_PERMRES_wise=(
    CUST_PERMRES_wise
    .withColumn(
        "Total_rate",
        ((f.col("good")+f.col("bad"))/(total_good_CUST_PERMRES_wise+total_bad_CUST_PERMRES_wise))
        )
    .withColumn(
        "good_rate",
        f.col("good")/total_good_CUST_PERMRES_wise    
    )
    .withColumn(
        "bad_rate",
        f.col("bad")/total_bad_CUST_PERMRES_wise   
    )
    .withColumn(
        "odd",
        f.col("good")/f.col("bad")  
    )
    .withColumn(
    "IV",
    (
        ((f.col("good") / total_good_CUST_PERMRES_wise) - (f.col("bad") / total_bad_CUST_PERMRES_wise)) *
        f.ln((f.col("good") / total_good_CUST_PERMRES_wise) / (f.col("bad") / total_bad_CUST_PERMRES_wise))
    )
)
     .withColumn(
       "good_rate_of_sub_cat",
    f.col("good") / (f.col("good") + f.col("bad"))
    )
    .orderBy("good_rate_of_sub_cat")
)
# IV_value_CUST_PERMRES= 0.001037 + 0.091448 + 0.002238 + 0.014116
IV_value_CUST_PERMRES=0.008507+0.003907+0.000012+0.003984

In [492]:
CUST_PERMRES_wise.toPandas().head()

,CUST_PERMRES,good,bad,Total_rate,good_rate,bad_rate,odd,IV,good_rate_of_sub_cat
0,With G S Certificate,403,114,0.072086,0.067663,0.093750,3.535088,0.008507,0.779497
1,With Parents/Spouse,1057,249,0.182097,0.177468,0.204770,4.244980,0.003907,0.809342
2,Rent,165,33,0.027607,0.027703,0.027138,5.000000,0.000012,0.833333
3,With Billing Proof,4331,820,0.718210,0.727166,0.674342,5.281707,0.003984,0.840808


In [493]:
CUST_PERMRES_wise.toPandas().to_csv("../output/last/cust_pre_wise_active3w.csv")

PermissionError: [Errno 13] Permission denied: '../output/last/cust_pre_wise_active3w.csv'

In [494]:
IV_value_CUST_PERMRES

0.01641

### Guarantor PreNises

In [495]:
master_table_new_Guarantor_PreNises=(
     master_table_new
     .filter(f.col("GUAR_PERMRES").isNotNull())
#     .filter((f.col("CUST_PERMRES")=="With G S Certificate")|
#             (f.col("CUST_PERMRES")=="Rent")|
#            (f.col("CUST_PERMRES")=="No Guarantors")|
#             (f.col("CUST_PERMRES")=="With Parents/spouse")|
#             (f.col("CUST_PERMRES")=="With Billing Proof")
            
           )

Guarantor_PreNises_wise= (
       master_table_new_Guarantor_PreNises
#     .filter(~f.col("Guarantor PreNises").isNull())
    .groupBy("GUAR_PERMRES")
        .agg(
            f.sum("good").alias("good"),
            f.sum("bad").alias("bad")
        )
    )
total_good_Guarantor_PreNises_wise=(
       Guarantor_PreNises_wise
        .agg(f.sum("good").alias("good"))
        .collect()[0]['good']
    )
total_bad_Guarantor_PreNises_wise= (
        CUST_PERMRES_wise
        .agg(f.sum("bad").alias("bad"))
        .collect()[0]['bad']
    )

Guarantor_PreNises_wise=(
    Guarantor_PreNises_wise
    .withColumn(
        "Total_rate",
        ((f.col("good")+f.col("bad"))/(total_good_Guarantor_PreNises_wise+total_bad_Guarantor_PreNises_wise))
        )
    .withColumn(
        "good_rate",
        f.col("good")/total_good_Guarantor_PreNises_wise    
    )
    .withColumn(
        "bad_rate",
        f.col("bad")/total_bad_Guarantor_PreNises_wise   
    )
    .withColumn(
        "odd",
        f.col("good")/f.col("bad")  
    )
    .withColumn(
    "IV",
    (
        ((f.col("good") / total_good_Guarantor_PreNises_wise) - (f.col("bad") / total_bad_Guarantor_PreNises_wise)) *
        f.ln((f.col("good") / total_good_Guarantor_PreNises_wise) / (f.col("bad") / total_bad_Guarantor_PreNises_wise))
    )
)
    .withColumn(
       "good_rate_of_sub_cat",
    f.col("good") / (f.col("good") + f.col("bad"))
    )
    .orderBy("good_rate_of_sub_cat")
)

# IV_Guarantor_PreNises_wise=4.694247e-08+5.527456e-05+2.444583e-04+1.225676e-03+3.686925e-05
IV_Guarantor_PreNises_wise=0.003354+0.009649+0.002598+0.000282+0.046818

In [496]:
Guarantor_PreNises_wise.toPandas()

,GUAR_PERMRES,good,bad,Total_rate,good_rate,bad_rate,odd,IV,good_rate_of_sub_cat
0,With G S Certificate,122,36,0.022036,0.020490,0.029605,3.388889,0.003354,0.772152
1,Rent,1039,265,0.181869,0.174505,0.217928,3.920755,0.009649,0.796779
2,With Parents/Spouse,952,220,0.163459,0.159893,0.180921,4.327273,0.002598,0.812287
3,With Billing Proof,2781,582,0.469038,0.467081,0.478618,4.778351,0.000282,0.826940
4,No Guaranter,1060,120,0.164575,0.178032,0.098684,8.833333,0.046818,0.898305


In [435]:
Guarantor_PreNises_wise.toPandas().to_csv("../output/last/guarantor_pre_wise_active3w.csv")

In [456]:
IV_Guarantor_PreNises_wise

0.062701

### Exposure Category(Score)

In [460]:
master_table_exp=(
    master_table_new.orderBy("EXP_CD")
    .filter(f.col("EXP_CD").isNotNull())
#     .filter(~f.col("Exposure").isNull())
#     .withColumn("Exposure_cat",
#                 f.when(f.col("Exposure").between(0,10),"0-10")
#                 .when(f.col("Exposure").between(11,20),"11-20")
#                 .when(f.col("Exposure").between(21,30),"21-30")
#                 .when(f.col("Exposure").between(31,40),"31-40")
#                 .when(f.col("Exposure").between(41,50),"41-50")
#                 .when(f.col("Exposure").between(51,60),"51-60")
#                 .when(f.col("Exposure").between(61,70),"61-70")
#                 .when(f.col("Exposure").between(71,80),"71-80")
#                 .when(f.col("Exposure")>80,"80>")
#                )
    .withColumn("Exposure_cat",
                f.when(f.col("EXP_CD").between(0,20),"0-20")
                .when(f.col("EXP_CD").between(21,40),"21-40")
                .when(f.col("EXP_CD").between(41,60),"41-60")
                .when(f.col("EXP_CD").between(61,80),"61-80")
                .when(f.col("EXP_CD")>=80,"80>")
               )
    .filter(f.col("Exposure_cat").isNotNull())
    
)

master_table_new1=master_table_exp.groupBy("Exposure_cat").agg(f.count("FAC NO"))

Exposure_Category_Score_wise= (
        master_table_exp.groupBy("Exposure_cat")
        .agg(
            f.sum("good").alias("good"),
            f.sum("bad").alias("bad")
        )
    )
total_good_Exposure_Category_Score_wise=(
      Exposure_Category_Score_wise
        .agg(f.sum("good").alias("good"))
        .collect()[0]['good']
    )
total_bad_Exposure_Category_Score_wise= (
       Exposure_Category_Score_wise
        .agg(f.sum("bad").alias("bad"))
        .collect()[0]['bad']
    )

Exposure_Category_Score_wise=(
    Exposure_Category_Score_wise
    .withColumnRenamed("Exposure Category(Score)","Exp")
    .withColumn(
        "Total_rate",
        ((f.col("good")+f.col("bad"))/(total_good_Exposure_Category_Score_wise+total_bad_Exposure_Category_Score_wise))
        )
    .withColumn(
        "good_rate",
        f.col("good")/total_good_Exposure_Category_Score_wise    
    )
    .withColumn(
        "bad_rate",
        f.col("bad")/total_bad_Exposure_Category_Score_wise  
    )
    .withColumn(
        "odd",
        f.col("good")/f.col("bad")  
    )
    .withColumn(
    "IV",
    (
        ((f.col("good") / total_good_Exposure_Category_Score_wise) - (f.col("bad") / total_bad_Exposure_Category_Score_wise)) *
        f.ln((f.col("good") / total_good_Exposure_Category_Score_wise) / (f.col("bad") / total_bad_Exposure_Category_Score_wise))
    )
)
    .withColumn(
       "good_rate_of_sub_cat",
    f.col("good") / (f.col("good") + f.col("bad"))
    )
    .orderBy("good_rate_of_sub_cat")
)
# IVP_Exp_value=(
# 0.050055+
# 0.030004+
# 0.055117+
# 0.001172+
# 0.053804+
# 0.039878+
# 0.034502+
# 0.007726+
# 0.003169
# )
# IVP_Exp_value=0.063524+0.018908+0.005051+0.048916+0.029908
IVP_EXP_value=0.101394+0.004872+0.033171+0.059999+	0.018468

In [461]:
master_table_new1.show()

+------------+-------------+
|Exposure_cat|count(FAC NO)|
+------------+-------------+
|       21-40|          782|
|        0-20|          153|
|       61-80|         2105|
|         80>|         2369|
|       41-60|         1373|
+------------+-------------+



In [462]:
Exposure_Category_Score_wise.toPandas().head(10)

,Exposure_cat,good,bad,Total_rate,good_rate,bad_rate,odd,IV,good_rate_of_sub_cat
0,80>,1764,605,0.349307,0.314270,0.517536,2.915702,0.101394,0.744618
1,61-80,1779,326,0.310380,0.316943,0.278871,5.457055,0.004872,0.845131
2,41-60,1210,163,0.202448,0.215571,0.139435,7.423313,0.033171,0.881282
3,21-40,717,65,0.115305,0.127739,0.055603,11.030769,0.059999,0.916880
4,0-20,143,10,0.022560,0.025477,0.008554,14.300000,0.018468,0.934641


In [421]:
Exposure_Category_Score_wise.toPandas().to_csv("../output/last/exp_cat_wise_active3w.csv")

In [463]:
IVP_EXP_value

0.217904

### DSCR

In [464]:
master_table_new_dcsr=(
     master_table_new
.filter(~f.col("DSCR_C").isNull())
    .withColumn("DSCR_cat",
                f.when(f.col("DSCR_C").between(0,5),"0-5")
                .when(f.col("DSCR_C").between(6,11),"6-11")
                .when(f.col("DSCR_C").between(12,17),"12-17")
                .when(f.col("DSCR_C").between(18,23),"18-23")
                .when(f.col("DSCR_C")>23,"23>")
               )
    .filter(f.col("DSCR_cat").isNotNull())
)
master_table_new_dcsr1=master_table_new_dcsr.groupBy("DSCR_cat").agg(f.count("DSCR_C"))
DSCR= (
      master_table_new_dcsr.groupBy("DSCR_cat")
        .agg(
            f.sum("good").alias("good"),
            f.sum("bad").alias("bad")
        )
    )
total_good_DSCR=(
       DSCR
        .agg(f.sum("good").alias("good"))
        .collect()[0]['good']
    )
total_bad_DSCR= (
        DSCR
        .agg(f.sum("bad").alias("bad"))
        .collect()[0]['bad']
    )

DSCR=(
    DSCR
    .withColumn(
        "Total_rate",
        ((f.col("good")+f.col("bad"))/(total_good_DSCR+total_bad_DSCR))
        )
    .withColumn(
        "good_rate",
        f.col("good")/total_good_DSCR   
    )
    .withColumn(
        "bad_rate",
        f.col("bad")/total_bad_DSCR 
    )
    .withColumn(
        "odd",
        f.col("good")/f.col("bad")  
    )
    .withColumn(
    "IV",
    (
        ((f.col("good") / total_good_DSCR) - (f.col("bad") / total_bad_DSCR)) *
        f.ln((f.col("good") / total_good_DSCR) / (f.col("bad") / total_bad_DSCR))
    )
)
    .withColumn(
       "good_rate_of_sub_cat",
    f.col("good") / (f.col("good") + f.col("bad"))
    )
    .orderBy("good_rate_of_sub_cat")
)
# IV_value_DSCR= 0.000052+0.004582+0.0036=66+0.023002+0.000092
IV_value_DSCR= 	0.004085+0.001120+	0.003951+0.019609+0.003596

In [465]:
master_table_new_dcsr1.show()

+--------+-------------+
|DSCR_cat|count(DSCR_C)|
+--------+-------------+
|   12-17|           70|
|     23>|          153|
|     0-5|         5652|
|    6-11|          750|
|   18-23|           39|
+--------+-------------+



In [466]:
DSCR.toPandas().head()

,DSCR_cat,good,bad,Total_rate,good_rate,bad_rate,odd,IV,good_rate_of_sub_cat
0,0-5,4620,1032,0.848139,0.837867,0.897391,4.476744,0.004085,0.817410
1,12-17,61,9,0.010504,0.011063,0.007826,6.777778,0.001120,0.871429
2,23>,135,18,0.022959,0.024483,0.015652,7.500000,0.003951,0.882353
3,6-11,662,88,0.112545,0.120058,0.076522,7.522727,0.019609,0.882667
4,18-23,36,3,0.005852,0.006529,0.002609,12.000000,0.003596,0.923077


In [430]:
DSCR.toPandas().to_csv("../output/last/dscr_cat_wise_active_3w.csv")

In [431]:
IV_value_DSCR

0.032361

IV (Information Value Statistic)	Description of Correlation
<0.01                               No Correlation
0.01 - 0.03                        	Marginal contribution
0.03 - 0.10	                        Adequate predictability
0.10 - 0.30	                        Strong predictor
0.30 - 0.50	                        Very Strong predictor
>=0.5	                            "Suspicious" - Too good to be true


In [467]:
print("IV_value_CUST_PERMRES: ",IV_value_CUST_PERMRES)
print("IV_Guarantor_PreNises_wise:",IV_Guarantor_PreNises_wise)
print("IV_value_DSCR:",IV_value_DSCR)
print("IVP_Exp_value:",IVP_EXP_value)
print("Iv_value_Product_wise:",Iv_value_Product_wise)

IV_value_CUST_PERMRES:  0.01641
IV_Guarantor_PreNises_wise: 0.062701
IV_value_DSCR: 0.032361
IVP_Exp_value: 0.217904
Iv_value_Product_wise: 0.08003299999999999


# Selecting Customer good or bad (On Total Collectiion Ratio)